In [3]:
pip install -q -U google-genai

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [22]:
from dotenv import load_dotenv
import minsearch
import json

In [8]:
with open('01-intro/documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [9]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [10]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [11]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [12]:
index.fit(documents)

In [13]:
from google import genai

# Load environment variables from .env file
load_dotenv()

client = genai.Client()

response = client.models.generate_content(
    model="gemini-2.0-flash", contents="Explain how AI works in a few words"
)
print(response.text)

AI learns from data to make predictions or decisions.



In [14]:
q = 'the course has already started, can I still enroll?'

In [15]:
response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=q
)
print(response.text)

Whether or not you can enroll in a course that has already started depends on a few factors:

*   **The institution/platform:** Each school, university, online learning platform (like Coursera, edX, Udemy, Skillshare, etc.) has its own policies.
*   **The course itself:** Some courses are designed to be self-paced, allowing enrollment at any time. Others have strict deadlines and require enrollment before the start date.
*   **The duration of the course:** If the course is a long one (e.g., a full semester), you might still be able to enroll within the first week or two, but it becomes less likely as more time passes. For shorter courses, it may already be too late.
*   **Instructor approval:** In some cases, even if the official deadline has passed, you might be able to enroll with the instructor's permission.

To find out, you should:

1.  **Check the course's official website or the platform's information page:** Look for enrollment deadlines, late enrollment policies, or contact in

In [16]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [17]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT does not contain the answer, output NONE

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [18]:
def llm(prompt):
    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=prompt
    )
    return response.text

In [ ]:
def rag(query):
    search_results = search(query)
    
    if not search_results:
        return "Sorry, I couldn't find any relevant information."
    
    prompt = build_prompt(query, search_results)
    
    answer = llm(prompt)
    
    return answer

In [21]:
rag('the course has already started, can I still enroll?')

"Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects, so don't leave everything for the last minute.\n"